In [1]:
import pandas as pd
data = pd.read_csv("dane.csv", delimiter=";", names=['From', 'To', 'Date', 'Cost'])

In [2]:
# data cleaning
data["To"] = data["To"].str.strip()
data['year'] = [int(i[0].split("-")[0]) for i in data['Date'].str.split()]
data['month'] = [int(i[0].split("-")[1]) for i in data['Date'].str.split()]
data['day'] = [int(i[0].split("-")[2]) for i in data['Date'].str.split()]
data['hour'] = [int(i[1].split(":")[0]) for i in data['Date'].str.split()]
data['minute'] = [int(i[1].split(":")[1]) for i in data['Date'].str.split()]
data['second'] = [int(i[1].split(":")[2]) for i in data['Date'].str.split()]
data = data.drop(["Date"], axis=1)

In [3]:
# LabelEncoder -> changes categorical data into numerical
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = data['To'].tolist()
labels = labels + data['From'].tolist()
labels = list(set(labels))
le.fit(labels)
data['From_edge'] = le.transform(data['From'])
data['To_edge'] = le.transform(data['To'])

In [21]:
# data from customer (tak tak powinno byc city... nie chcialem juz zmieniac)
max_cost = 300
max_cities = 5
time_in_citie_in_days = 2
start_citie = "KRK"
year = 2019
month = 1
day = 10

In [5]:
data['route_id'] = data.index

In [6]:
# add route_id
for i, z in enumerate(list(set((data['From'] + data['To']).tolist()))):
    data.loc[(data['From'] == z[:3]) & (data['To'] == z[3:]), ['route_id']] = i

In [7]:
data.head()

,From,To,Cost,year,month,day,hour,minute,second,From_edge,To_edge,route_id
0,KBP,WRO,36,2019,1,10,16,10,0,175,390,3727
1,KBP,VNO,38,2019,1,10,10,25,0,175,384,2798
2,KBP,VIN,41,2019,1,10,21,25,0,175,379,555
3,KBP,KRK,41,2019,1,10,21,40,0,175,186,753
4,KBP,CPH,42,2019,1,10,21,50,0,175,78,2563


In [8]:
# create dataframe only with start city and all the connections from it
stage_1 = data.loc[(data['From'] == 'KRK') & (data['year'] == year) & (data['month'] == month) & (data['day'] == day) & (data['Cost'] <= max_cost)]

In [22]:
# create rest dataframes (with transit cities)
stages = [stage_1]
for stage in range(max_cities-1):
    day = day+time_in_citie_in_days
    stages.append(data.loc[(data['From'].isin(stages[-1]['To'].values)) & ~(data['route_id'].isin(stages[-1]['route_id'].values)) & (data['To'] != start_citie) & (data['year'] == year) & (data['month'] == month) & (data['day'] == day) & (data['Cost'] <= max_cost)])
    
    for i in range(len(stages)-1):
        stages[-1] = stages[-1].loc[(~stages[-1]['route_id'].isin(stages[i]['route_id'].values))]
stages.append(data.loc[(data['From'].isin(stages[-1]['To'].values)) & (data['To'] == start_citie) & (data['year'] == year) & (data['month'] == month) & (data['day'] == day+time_in_citie_in_days) & (data['Cost'] <= max_cost)])

In [23]:
for i in stages:
    print(i.shape)

(25, 12)
(750, 12)
(1895, 12)
(601, 12)
(235, 12)
(20, 12)


In [24]:
# make correction to cities numbers (ogolnie powinno byc bardziej From_vertex, To_vertex)
import numpy as np
tmp = 0
err = 0
for i, stage in enumerate(stages):
    if i != 0:
        err = tmp - np.max(stages[i-1]['From_edge'])
        stage['From_edge'] = le.transform(stage['From'].tolist())
        stage['From_edge'] += (np.max(stages[i-1]['From_edge']) + 1 + err)
        stage['To_edge'] = le.transform(stage['To'].tolist())
        stage['To_edge'] += (np.max(stages[i-1]['To_edge']) + 1)
        tmp = np.max(stages[i-1]['To_edge'])
        stages[i] = stage
    else:
        stage['From_edge'] = 0
        stage['To_edge'] = le.transform(stage['To'].tolist())
        stage['To_edge'] += 1
        stages[i] = stage

/home/amadeusz/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/amadeusz/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/amadeusz/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [25]:
# set index numbers as link numbers
length = 0
for i, stage in enumerate(stages):
    if i == 0:
        stage.index = stage.reset_index().drop(['index'], axis=1).index + 1
        length = len(stage)
    else:
        stage.index = stage.reset_index().drop(['index'], axis=1).index + 1 + length
        length = length + len(stage)

In [26]:
stages[-1].head()

,From,To,Cost,year,month,day,hour,minute,second,From_edge,To_edge,route_id
3507,HEL,KRK,54,2019,1,20,16,10,0,1727,2164,676
3508,VIE,KRK,18,2019,1,20,13,25,0,1959,2164,1868
3509,BCN,KRK,41,2019,1,20,16,15,0,1608,2164,1768
3510,CRL,KRK,15,2019,1,20,13,25,0,1662,2164,803
3511,STN,KRK,33,2019,1,20,19,0,0,1912,2164,3362


Save output as data.dat file

In [27]:
with open("data.dat", "a") as f:
    f.write("""
data;

param V_count := {V_count};
param E_count := {E_count};
param D_count := {D_count};

param : h  s t :=
 1      1 {s} {t}
;
        """.format(V_count=stages[-1]['To_edge'].iloc[0], 
                   E_count=stages[-1].tail(1).index.values[0], 
                   D_count=1, 
                   s=0, 
                   t=stages[-1]['To_edge'].iloc[0]))

In [28]:
with open("data.dat", "a") as f:
     f.write("""
param : A :=
""")

In [29]:
with open("data.dat", "a") as f:
    for stage in stages:
        for val, index in zip(stage.values, stage.index.values):
            f.write("""
 {link_number},{to_edge_number}    1
""".format(link_number=index, 
           to_edge_number=val[-3]))

In [30]:
with open("data.dat", "a") as f:
     f.write("""
;
""")

In [31]:
with open("data.dat", "a") as f:
     f.write("""
param : B :=
""")

In [32]:
with open("data.dat", "a") as f:
    for stage in stages:
        for val, index in zip(stage.values, stage.index.values):
            f.write("""
 {link_number},{to_edge_number}    1
""".format(link_number=index, 
           to_edge_number=val[-2]))

In [33]:
with open("data.dat", "a") as f:
     f.write("""
;
""")

In [34]:
with open("data.dat", "a") as f:
     f.write("""
param : KSI :=
""")

In [35]:
with open("data.dat", "a") as f:
    for stage in stages:
        for val, index in zip(stage.values, stage.index.values):
            f.write("""
 {link_number}    {cost}
""".format(link_number=index, 
           cost=val[2]))

In [36]:
with open("data.dat", "a") as f:
     f.write("""
;
""")

In [37]:
with open("data.dat", "a") as f:
     f.write("""
end;
""")

In [38]:
stages[0].loc[2]

From          KRK
To            SXF
Cost           11
year         2019
month           1
day            10
hour            9
minute         40
second          0
From_edge       0
To_edge       338
route_id     3335
Name: 2, dtype: object

In [39]:
stages[1].loc[537]

From          SXF
To            VLC
Cost           15
year         2019
month           1
day            12
hour           17
minute         15
second          0
From_edge     338
To_edge       772
route_id     2138
Name: 537, dtype: object

In [40]:
stages[2].loc[1996]

From          VLC
To            FRA
Cost           12
year         2019
month           1
day            14
hour           17
minute         35
second          0
From_edge     772
To_edge       912
route_id     1752
Name: 1996, dtype: object

In [41]:
stages[3].loc[2779]

From          FRA
To            STN
Cost           15
year         2019
month           1
day            16
hour           19
minute          0
second          0
From_edge     912
To_edge      1515
route_id     4011
Name: 2779, dtype: object

In [42]:
stages[4].loc[3301]

From          STN
To            BGY
Cost           20
year         2019
month           1
day            18
hour           14
minute         40
second          0
From_edge    1515
To_edge      1615
route_id     1365
Name: 3301, dtype: object

In [43]:
stages[5].loc[3514]

From          BGY
To            KRK
Cost           20
year         2019
month           1
day            20
hour           15
minute         35
second          0
From_edge    1615
To_edge      2164
route_id     2465
Name: 3514, dtype: object